In [1]:
PATH='../../../data/aclImdb/'

TRN_PATH='train/all/'
VAL_PATH='test/all/'
TRN=f'{PATH}{TRN_PATH}'
VAL=f'{PATH}{VAL_PATH}'

%ls {PATH}

README  imdb.vocab  imdbEr.txt  models/  test/  tmp/  train/


In [2]:
trn_files = !ls {TRN}

In [3]:
trn_files[:10]

['0_0.txt',
 '0_3.txt',
 '0_9.txt',
 '10000_0.txt',
 '10000_4.txt',
 '10000_8.txt',
 '10001_0.txt',
 '10001_10.txt',
 '10001_4.txt',
 '10002_0.txt']

In [4]:
reviews=!cat {TRN}{trn_files[6]}

In [5]:
reviews[0]

"Everybody has seen 'Back To The Future,' right? Whether you LIKE that movie or not, you've seen an example of how to make a time-travel movie work. A torn-up poster for 'Back To The Future' shows up in this movie, representing, perhaps unintentionally, what the makers of 'Tangents' (aka 'Time Chasers') did to the time-travel formula. Then again, the movie claims to have been made in 1994, but it looks -- and sounds -- like it was produced at least ten years earlier, so maybe they achieved time-travel after all.<br /><br />Start with an intensely unappealing leading man. I mean, what woman doesn't love gangly, whiny, lantern-jawed, butt-chinned, mullet-men with Coke-bottle glasses? Oh, none of you? Prepare to tough it out, ladies, cuz that's what this movie gives you.<br /><br />Second, add a leading lady who -- while not entirely unattractive -- represents many '80s clich��s: big hair, too much makeup, two different plaids, shoulder pads, acid-washed mom-jeans, etc.<br /><br />Throw i

In [6]:
!find {TRN} -name '*.txt' | xargs cat | wc -w

17486270


In [7]:
!find {VAL} -name '*.txt' | xargs cat | wc -w

5686609


In [8]:
import spacy

In [9]:
spacy_tok = spacy.load('en')

In [10]:
' '.join([word.string.strip() for word in spacy_tok(reviews[0])])

"Everybody has seen ' Back To The Future , ' right ? Whether you LIKE that movie or not , you 've seen an example of how to make a time - travel movie work . A torn - up poster for ' Back To The Future ' shows up in this movie , representing , perhaps unintentionally , what the makers of ' Tangents ' ( aka ' Time Chasers ' ) did to the time - travel formula . Then again , the movie claims to have been made in 1994 , but it looks -- and sounds -- like it was produced at least ten years earlier , so maybe they achieved time - travel after all.<br /><br />Start with an intensely unappealing leading man . I mean , what woman does n't love gangly , whiny , lantern - jawed , butt - chinned , mullet - men with Coke - bottle glasses ? Oh , none of you ? Prepare to tough it out , ladies , cuz that 's what this movie gives you.<br /><br />Second , add a leading lady who -- while not entirely unattractive -- represents many ' 80s clich � � s : big hair , too much makeup , two different plaids , s

In [11]:
import torchtext
from torchtext import data, vocab
from torchtext.datasets import language_modeling

In [12]:
TEXT = data.Field(lower = True, tokenize = "spacy")

In [13]:
FILES = dict(train=TRN_PATH, validation=VAL_PATH, test=VAL_PATH)

In [34]:
bs=32; bptt=70;

In [15]:
from fastai.nlp import *

/home/ngj/anaconda3/envs/fastai/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [ ]:
langmodel = LanguageModelData.from_text_files(PATH,TEXT,**FILES,bs=bs,bptt=bptt,min_freq=10);

In [17]:
import dill as pickle

In [18]:
!ls ../../../data/aclImdb/

README	imdb.vocab  imdbEr.txt	models	test  tmp  train


In [19]:
!cd ../../../data/aclImdb && mkdir models

mkdir: cannot create directory 'models': File exists


In [20]:
!cd ../../../data/aclImdb/models && touch TEXT.pkl


!cd ../../../data/aclImdb/ &&  mkdir models

In [21]:
pickle.dump(TEXT, open(f'{PATH}models/TEXT.pkl','wb'))

In [22]:
len(langmodel.trn_dl), langmodel.nt, len(langmodel.trn_ds), len(langmodel.trn_ds[0].text)

(4278, 37392, 1, 20540756)

In [23]:
TEXT.vocab.itos[2:12]

['the', ',', '.', 'and', 'a', 'of', 'to', 'is', 'in', 'it']

In [24]:
TEXT.vocab.stoi['the']

2

In [25]:
langmodel.trn_ds[0].text[:12]

['i',
 'can',
 'not',
 'believe',
 'all',
 'of',
 'the',
 'bad',
 'reviews',
 'that',
 'this',
 'movie']

In [26]:
next(iter(langmodel.trn_dl))

(Variable containing:
     12   3338    368  ...    4298     33   1716
     72    828     13  ...       5    282    702
     31      7     77  ...    4632      8     60
         ...            ⋱           ...         
     76      2      4  ...   11621  10561  16744
    219   6639     43  ...       4     30      4
   1611      3   2186  ...     330      5     12
 [torch.cuda.LongTensor of size 74x64 (GPU 0)], Variable containing:
     72
    828
     13
   ⋮   
      3
      0
     97
 [torch.cuda.LongTensor of size 4736 (GPU 0)])

In [27]:
em_sz=200
hidden=500
layers=3

In [28]:
opt_fun=partial(optim.Adam, betas=(0.7,0.99))

In [29]:
from fastai.learner import *

In [30]:
learner = langmodel.get_model(opt_fun,em_sz,hidden,layers,dropouti=0.05,dropout=0.05,wdrop=0.1,dropoute=0.02,dropouth=0.05)

In [31]:
learner.reg_fun=partial(seq2seq_reg, alpha=2, beta=1)

In [32]:
learner.clip=0.3

In [33]:
learner.fit(3e-3, 4, cycle_len=1, wds=1e-6)

  0%|          | 0/4278 [00:00<?, ?it/s]


RuntimeError: cuda runtime error (2) : out of memory at /opt/conda/conda-bld/pytorch_1518244421288/work/torch/lib/THC/generic/THCStorage.cu:58